In [1]:
import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=RRuntimeWarning)
import sys
import os

In [2]:
import pandas as pd
import numpy as np

In [3]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [4]:
# Enable R magic
%load_ext rpy2.ipython

In [5]:
HERE = os.path.dirname(os.path.realpath('__file__'))
PROJECT = os.path.abspath(os.path.join(HERE, '..'))
DATA = os.path.join(PROJECT, 'data')

GENOTYPE_DATA = os.path.join(DATA, 'Asif_Genotype_Disease_Only_ROSMAP.CSV')
SUGBRAPH_SNPS = os.path.join(DATA, 'subgraphs15_snps_mod1.csv')

In [6]:
%%R

library(h2o)
library(randomForest)
library(glmnet)
library(gbm)
library(bnlearn)
library(ggplot2)
library(CORElearn)

R[write to console]: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


R[write to console]: 
Attache Paket: ‘h2o’


R[write to console]: The following objects are masked from ‘package:stats’:

    cor, sd, var


R[write to console]: The following objects are masked from ‘package:base’:

    %*%, %in%, &&, apply, as.factor, as.numeric, colnames, colnames<-,
    ifelse, is.character, is.factor, is.numeric, log, log10, log1p,
    log2, round, signif, trunc, ||


R[write to console]: randomForest 4.6-14

R[write to console]: Type rfNews() to see new features/changes/bug fixes.

R[write to console]: Lade nötiges Paket: Matrix

R[write to console]: Loaded glmnet 3.0-2


R[write

In [7]:
%%R

#### main function ####

ann_fun = function(input_data){
  if(dim(input_data)[2] > 1)
  {
    h2o.init(nthreads = -1, max_mem_size = "12g")
    
    #sc = scale(input_data)
    myx = as.h2o(scale(input_data)) 
    
    n = round(dim(input_data)[2])
    m = round(dim(input_data)[2]/2)
    
    hyper_params <- list(hidden = list(1, c(n, 1), c(n,m,1), c(m,1)),
                         input_dropout_ratio = c(0,0.05, 0.2,0.5),
                         l2=10^c(-4:4))
    
    
    r = sample(20:60000, 1)
    grid = h2o.grid("deeplearning", 
                    grid_id = paste("mygrid", r, sep="_"), 
                    autoencoder = TRUE,
                    x = colnames(myx),
                    training_frame = myx, 
                    seed=1234567, 
                    stopping_metric="MSE", 
                    stopping_rounds = 5,
                    #activation= "TanhWithDropout",
                    activation= "Tanh",
                    standardize=TRUE,
                    epochs=500,
                    hyper_params = hyper_params)
    
    gbm_sorted_grid <- h2o.getGrid(grid_id = paste("mygrid", r, sep="_"), sort_by = "mse")
    fit <- h2o.getModel(gbm_sorted_grid@model_ids[[1]])
    #nlayers = length(strsplit(gbm_sorted_grid@summary_table[1,1], ",")[[1]])
    #browser()
    nlayers = length(strsplit(substr(gbm_sorted_grid@summary_table[1,1], 2, nchar(gbm_sorted_grid@summary_table[1,1])-1), ",")[[1]])
    newvar = as.data.frame(h2o.deepfeatures(fit, myx, nlayers))
    
    #Rename column
    newvar = as.data.frame(newvar[,1])
    colnames(newvar) = paste( sub("_.*", '', colnames(input_data)[1]) , sub(".*_", '',colnames(input_data)[1]) , sep= "_")    
    
    #save meta-features in variable 
    # need to change the nrow depending on number of samples(patients)
    subgraph_feature = data.frame(matrix(NA, nrow = 540, ncol = 1)) #540
    colnames(subgraph_feature) = "dummy"
    
    subgraph_feature = cbind(subgraph_feature,newvar) 
    subgraph_feature$dummy = NULL
    
  }else if(dim(input_data)[2] == 1 ){
    subgraph_feature = as.data.frame(input_data)
    colnames(newvar) = paste( sub("_.*", '', colnames(input_data)[1]) , sub(".*_", '',colnames(input_data)[1]) , sep= "_")
  }else{
    # need to change the nrow depending on number of samples(patients)
    subgraph_feature = data.frame(matrix(NA, nrow = 540, ncol = 1)) #540
    colnames(subgraph_feature) = "dummy"
  }
  outcome = list(subgraph_feature, fit) # scales=attributes(sc)
  return(outcome)
  #return(subgraph_feature)
}


In [8]:
snps_data = pd.read_csv(GENOTYPE_DATA, index_col=0)
snps_data = snps_data.transpose()

In [9]:
snps_data.head()

SNP,rs7539455,rs6687381,rs4846028,rs12095517,rs4073574,rs4845875,rs4845877,rs4845881,rs4846048,rs6541003,...,rs7815138,rs1877829,rs2221222,rs6471533,rs7011044,rs2318878,rs2318876,rs3104965,rs2873824,rs4524756
MAP01797756,0,1,0,1,0,1,1,1,1,1,...,1,1,0,0,0,1,0,0,2,0
MAP05522533,1,0,0,1,0,1,1,1,1,1,...,0,0,0,1,1,1,0,1,1,1
MAP07265221,1,0,1,0,0,0,0,0,0,0,...,1,0,1,1,1,0,1,0,0,1
MAP21362537,0,1,1,0,0,0,0,0,0,1,...,2,1,0,0,1,0,0,0,0,2
MAP24073245,1,0,0,1,1,2,2,1,1,2,...,2,2,0,2,0,1,0,0,0,0


In [10]:
snps_data.shape

(194, 148)

In [11]:
snps_list = pd.read_csv(
    SUGBRAPH_SNPS,header=None, index_col=0
)

subgraph2snps = {
    index: snps.dropna().tolist()
    for index, snps in snps_list.iterrows()
}

Dictionary of subgraphs to dataframe slices for the SNPs in the subgraph

In [12]:
snp_mat = {
    subgraph: snps_data[snps]
    for subgraph, snps in subgraph2snps.items()
}

In [24]:
subgraph_4 = pandas2ri.py2rpy_pandasdataframe(snp_mat['Subgraph_4'])

In [25]:
subgraph_4

rs1965869,rs11097200,rs11735482,...,rs1877829,rs11126836,rs1866561
...,...,...,...,...,...,...


In [30]:
%%R -i=subgraph_4
# see: https://ipython.org/ipython-doc/2/config/extensions/rmagic.html

# apply main function on each mechanisms matrices

#print(subgraph_4)

model.store <- ann_fun(subgraph_4)

####### This does not work to apply for each subgraph #######
#model.store <- lapply(subgraph_4, function(x) ann_fun(x))
save(model.store, file = "model.store.rosmapIdibaps-148_4.RData")


# autoen=matrix(nrow = 540, ncol = 15) # 358 PD patients and 15 subgraphs

# j=1
# for (i in model.store){
#   autoen[,j]=i[[1]][[1]]
#   j=j+1
# }

# rownames(autoen) <- rownames(snp.mat[[1]])
# colnames(autoen) <- names(model.store)
# save(autoen, file = "/home/memon/genetic_analyses/ann/output/subgraph15/autoen_matrix_rosmapIdibaps-148_4.RData")


 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         2 minutes 7 seconds 
    H2O cluster timezone:       Europe/Berlin 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.26.0.2 
    H2O cluster version age:    5 months and 22 days !!! 
    H2O cluster name:           H2O_started_from_R_danieldomingo_uwg341 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   10.50 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.6.2 (2019-12-12) 

  |======================================================================| 100%
  |========================================================

Hyper-parameter: hidden, 6
 Hyper-parameter: hidden, 1
Hyper-parameter: input_dropout_ratio, 0.5
Hyper-parameter: l2, 100
[2020-01-18 17:53:40] failure_details: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably large or lead to large activation values.
Try a different initial distribution, a bounded activation function (Tanh), adding regularization
(via max_w2, l1, l2, dropout) or learning rate (either enable adaptive_rate or use a smaller learning rate or faster annealing).

For more information visit:
  http://jira.h2o.ai/browse/TN-4' 
[2020-01-18 17:53:40] failure_stack_traces: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably larg

Hyper-parameter: hidden, 12
 Hyper-parameter: hidden, 1
Hyper-parameter: input_dropout_ratio, 0
Hyper-parameter: l2, 1000
[2020-01-18 17:53:40] failure_details: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably large or lead to large activation values.
Try a different initial distribution, a bounded activation function (Tanh), adding regularization
(via max_w2, l1, l2, dropout) or learning rate (either enable adaptive_rate or use a smaller learning rate or faster annealing).

For more information visit:
  http://jira.h2o.ai/browse/TN-4' 
[2020-01-18 17:53:40] failure_stack_traces: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably larg

Hyper-parameter: hidden, 12
 Hyper-parameter: hidden, 1
Hyper-parameter: input_dropout_ratio, 0.05
Hyper-parameter: l2, 10
[2020-01-18 17:53:40] failure_details: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably large or lead to large activation values.
Try a different initial distribution, a bounded activation function (Tanh), adding regularization
(via max_w2, l1, l2, dropout) or learning rate (either enable adaptive_rate or use a smaller learning rate or faster annealing).

For more information visit:
  http://jira.h2o.ai/browse/TN-4' 
[2020-01-18 17:53:40] failure_stack_traces: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably lar

Hyper-parameter: hidden, 6
 Hyper-parameter: hidden, 1
Hyper-parameter: input_dropout_ratio, 0.2
Hyper-parameter: l2, 10
[2020-01-18 17:53:40] failure_details: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably large or lead to large activation values.
Try a different initial distribution, a bounded activation function (Tanh), adding regularization
(via max_w2, l1, l2, dropout) or learning rate (either enable adaptive_rate or use a smaller learning rate or faster annealing).

For more information visit:
  http://jira.h2o.ai/browse/TN-4' 
[2020-01-18 17:53:40] failure_stack_traces: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably large

...

Hyper-parameter: hidden, 6
 Hyper-parameter: hidden, 1
Hyper-parameter: input_dropout_ratio, 0.2
Hyper-parameter: l2, 100
[2020-01-18 17:53:41] failure_details: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably large or lead to large activation values.
Try a different initial distribution, a bounded activation function (Tanh), adding regularization
(via max_w2, l1, l2, dropout) or learning rate (either enable adaptive_rate or use a smaller learning rate or faster annealing).

For more information visit:
  http://jira.h2o.ai/browse/TN-4' 
[2020-01-18 17:53:41] failure_stack_traces: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably larg

Hyper-parameter: hidden, 6
 Hyper-parameter: hidden, 1
Hyper-parameter: input_dropout_ratio, 0.2
Hyper-parameter: l2, 1000
[2020-01-18 17:53:41] failure_details: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably large or lead to large activation values.
Try a different initial distribution, a bounded activation function (Tanh), adding regularization
(via max_w2, l1, l2, dropout) or learning rate (either enable adaptive_rate or use a smaller learning rate or faster annealing).

For more information visit:
  http://jira.h2o.ai/browse/TN-4' 
[2020-01-18 17:53:41] failure_stack_traces: DistributedException from localhost/127.0.0.1:54321: '

Trying to predict with an unstable model.
Job was aborted due to observed numerical instability (exponential growth).
Either the weights or the bias values are unreasonably lar

  |======================================================================| 100%


R[write to console]: Fehler in data.frame(..., check.names = FALSE) : 
  Argumente implizieren unterschiedliche Anzahl Zeilen: 540, 194
Ruft auf: <Anonymous> ... withVisible -> ann_fun -> cbind -> cbind -> data.frame

R[write to console]: Además: 
R[write to console]: Warnmeldung:

R[write to console]: In h2o.clusterInfo() :
R[write to console]:  
Your H2O cluster version is too old (5 months and 22 days)!
Please download and install the latest version from http://h2o.ai/download/




Fehler in data.frame(..., check.names = FALSE) : 
  Argumente implizieren unterschiedliche Anzahl Zeilen: 540, 194
Ruft auf: <Anonymous> ... withVisible -> ann_fun -> cbind -> cbind -> data.frame


In [ ]:
%%R
#' save models in h2o format

an_model <- lapply(model.store, '[[',2)  # This returns a list with only the 2nd element (h2o models)

for (i in an_model){
  h2o.saveModel(i, path = "/home/memon/genetic_analyses/ann/output/subgraph15/autoen_rosmapIdibaps-148_h2omodel_4/")
}

h2o.shutdown(prompt = FALSE)
gc()


In [ ]:
%%R 

fit = lm(mtcars$cyl ~ mtcars$mpg)
coefficients <- coef(fit)


In [ ]:
%%R
summary(fit)